# Get Copernicus DEM using EODAG

This tutorial will help you to search and download Copernicus DEM using EODAG

In [1]:
import os
from eodag import EODataAccessGateway

Create a workspace directory `eodag_workspace_copdem` where all our files will live:

In [2]:
workspace = 'eodag_workspace_copdem'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

## Search products

Init EODAG and get available product types from Copernicus DEM

In [3]:
dag = EODataAccessGateway()

dag.guess_product_type(keywords="DEM")

['COP_DEM_GLO30_DGED',
 'COP_DEM_GLO30_DTED',
 'COP_DEM_GLO90_DGED',
 'COP_DEM_GLO90_DTED']

Check which providers support `COP_DEM_GLO30_DGED`

In [4]:
product_type = "COP_DEM_GLO30_DGED"
dag.available_providers(product_type)

['creodias', 'wekeo']

In [5]:
search_geom = [0.25, 43.2, 2.8, 43.9]
search_result = dag.search(productType=product_type, geom=search_geom)
search_result

Product type 'COP_DEM_GLO30_DGED' is not available with provider 'peps'. Searching it on provider 'creodias' instead.


SearchResult([EOProduct(id=DEM1_SAR_DGE_30_20110216T174143_20140906T174215_ADS_000000_jSLP, provider=creodias),
              EOProduct(id=DEM1_SAR_DGE_30_20110417T175032_20140906T174215_ADS_000000_PIJx, provider=creodias),
              EOProduct(id=DEM1_SAR_DGE_30_20110428T174952_20140831T175054_ADS_000000_Q3FX, provider=creodias)])

Draw results geometries on a map:

In [6]:
import folium

# Create a map zoomed over the search area
fmap = folium.Map([43.5, 1.5], zoom_start=8)
# Create a layer that represents the search area in red
folium.Rectangle(
    bounds=[search_geom[0:2][::-1], search_geom[2:4][::-1]],
    color="red",
    tooltip="Search extent"
).add_to(fmap)
# Create a layer that maps the products found
folium.GeoJson(
    data=search_result,  # SearchResult has a __geo_interface__ interface used by folium to get its GeoJSON representation
    tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)
fmap


## Download the DEM files to workspace

Make sure credentials were set for the provider. Here we will download on `creodias` that needs an additionnal [TOTP](https://eodag.readthedocs.io/en/latest/getting_started_guide/configure.html?#use-otp-through-python-code) key:

In [7]:
dag.update_providers_config(
   """
   creodias:
      auth:
         credentials:
            totp: PLEASE_CHANGE_ME
   """
)

In [8]:
paths = dag.download_all(
    search_result, 
    output_dir=workspace,
)
paths

Downloaded products:   0%|          | 0/3 [00:00<?, ?product/s]

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

['/home/anesson/workspace/EODAG/dev/eodag/docs/notebooks/tutos/eodag_workspace_copdem/DEM1_SAR_DGE_30_20110216T174143_20140906T174215_ADS_000000_jSLP',
 '/home/anesson/workspace/EODAG/dev/eodag/docs/notebooks/tutos/eodag_workspace_copdem/DEM1_SAR_DGE_30_20110428T174952_20140831T175054_ADS_000000_Q3FX',
 '/home/anesson/workspace/EODAG/dev/eodag/docs/notebooks/tutos/eodag_workspace_copdem/DEM1_SAR_DGE_30_20110417T175032_20140906T174215_ADS_000000_PIJx']